# Distance to edge tool 

This code creates a mathematical edge of the tumor using dapi position data, then calculates the shortest distance to the border from each dapi position adding the results as a new column. Data used must contain X and Y dapi postion.

In [1]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull

# 1. Find all CSV files in the folder
csv_files = glob.glob('csv_for_tools/*.csv')

# 2. Error if none found
if not csv_files:
    raise FileNotFoundError("No CSV files found in 'csv_for_tools/'")

# 3. Pick the first CSV
first_csv = csv_files[0]
print(f"Reading: {first_csv}")

# 4. Read into a DataFrame
df = pd.read_csv(first_csv)

# (Optional) Inspect the top of the DataFrame
df.head()


Reading: csv_for_tools/merged_for_tools.csv


,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,ID,UNNAMED: 8,F480,LY6G,CD3,NK1.1,CD4,MHCII OFF F480,CD206,CD11B,CD8,MHCII
0,57597.965,47750.762,17,µm,Surface,Position,1,354843,0,1,0,0,0,0,0,0,0,0,0
1,58191.477,47161.969,14,µm,Surface,Position,1,354844,0,0,0,0,0,0,0,0,0,0,0
2,57969.219,47377.934,17,µm,Surface,Position,1,354845,0,0,0,0,0,0,0,0,0,0,0
3,57126.000,48221.410,12,µm,Surface,Position,1,354846,0,0,0,0,0,0,0,0,0,0,0
4,58455.406,46896.344,12,µm,Surface,Position,1,354847,0,0,0,0,0,0,0,0,0,0,0


# Check data before contuining

Make sure there is a positon x and position y column otherwise code will not work that is all. 

In [2]:

pts = df[['POSITION X', 'POSITION Y']].values             # shape (N,2) array

# 2) Compute the convex hull
hull = ConvexHull(pts)

# hull.simplices is an (E,2) array of index pairs (i, j)
# each representing an edge between pts[i] and pts[j].
edges = hull.simplices

# 3) Define a vectorized distance‐to‐segment function
def point_to_segment_distances(points, seg_start, seg_end):
    """
    points: (N,2) array
    seg_start, seg_end: each (2,) array
    returns: (N,) array of distances from each point to the segment
    """
    v = seg_end - seg_start                      # (2,)
    w = points - seg_start                       # (N,2)
    # projection factor of each point onto the infinite line
    t = np.einsum('ij,j->i', w, v) / np.dot(v, v)
    # clamp to [0,1] so we “snap” to the segment
    t_clamped = np.clip(t, 0.0, 1.0)             # (N,)
    proj = seg_start + t_clamped[:,None] * v     # (N,2)
    # euclidean distance from each point to its projection
    return np.linalg.norm(points - proj, axis=1)

# 4) Loop over edges, keep the minimum distance per point
all_dists = np.full(len(pts), np.inf)
for i, j in edges:
    start, end = pts[i], pts[j]
    d = point_to_segment_distances(pts, start, end)
    all_dists = np.minimum(all_dists, d)

# 5) Attach back to your DataFrame
df['Distance_to_Edge'] = all_dists

# 6) (Optional) Inspect
df

,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,ID,UNNAMED: 8,F480,LY6G,CD3,NK1.1,CD4,MHCII OFF F480,CD206,CD11B,CD8,MHCII,Distance_to_Edge
0,57597.965,47750.762,17,µm,Surface,Position,1,354843,0,1,0,0,0,0,0,0,0,0,0,525.085353
1,58191.477,47161.969,14,µm,Surface,Position,1,354844,0,0,0,0,0,0,0,0,0,0,0,291.570368
2,57969.219,47377.934,17,µm,Surface,Position,1,354845,0,0,0,0,0,0,0,0,0,0,0,417.954414
3,57126.000,48221.410,12,µm,Surface,Position,1,354846,0,0,0,0,0,0,0,0,0,0,0,273.334408
4,58455.406,46896.344,12,µm,Surface,Position,1,354847,0,0,0,0,0,0,0,0,0,0,0,144.888975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494200,51109.797,41862.387,15,µm,Surface,Position,1,852768,0,1,0,0,0,0,0,0,1,0,0,380.077926
494201,51540.227,41427.938,15,µm,Surface,Position,1,852769,0,0,0,0,0,0,0,0,1,0,0,493.394661
494202,52421.312,40542.230,11,µm,Surface,Position,1,852770,0,1,0,0,0,0,0,0,1,0,0,585.914989
494203,51111.664,41854.121,16,µm,Surface,Position,1,852771,0,0,1,0,0,0,0,0,1,0,0,378.827975


# Review the ouput above before continuing

If the output above looks correct then move to next cell below. Running the next cell will update the csv file and rewrite it by adding the edge distance column make sure you are ok with this before running next cell. 

In [3]:
csv_path = glob.glob('csv_for_tools/*.csv')[0]

df.to_csv(csv_path, index=False)